In [1]:
import pandas as pd
import numpy as np
import os
import json
import random

business = os.path.join("data", "business.csv")
economy = os.path.join("data", "economy.csv")
business_df = pd.read_csv(business)
economy_df = pd.read_csv(economy)

In [2]:
# Create copy of original DFs so that we do not make any irreversible changes 
business_copied = business_df.copy()
economy_copied = economy_df.copy()

In [3]:
# Uncleaned business data
business_copied.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [4]:
# Uncleaned economy data
economy_copied.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [5]:
# No imputations are necessary
economy_copied.isnull().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

In [6]:
# No imputations are necessary
business_copied.isnull().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

In [7]:
# Combine the two DFs together, and create new column for class with values
# "economy" and "business" which corresponds to each flight
business_copied["class"] = "business"
economy_copied["class"] = "economy"

cleaned = pd.concat([business_copied, economy_copied])

In [8]:
cleaned.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",business
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",business
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220",business
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450",business
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690",business


In [9]:
# Cleaned departure
cleaned["departure"] = pd.to_datetime(cleaned["date"] + " " + cleaned["dep_time"])

In [10]:
# Cleaned arrival
cleaned["arrival"] = pd.to_datetime(cleaned["date"] + " " + cleaned["arr_time"])

In [11]:
# Cleaned duration 
cleaned["duration"] = pd.to_timedelta(("0" + cleaned["time_taken"].str.split().str[0]) + " " + ("0" + cleaned["time_taken"].str.split().str[1]))
cleaned["duration"] = cleaned["duration"].dt.total_seconds()

In [12]:
# Cleaned Stop
cleaned["stops"] = (
    cleaned["stop"]
    .str.replace("\W+", "", regex=True)
    .apply(lambda x: "1stop" if "1stop" in x else x)
    .replace({"nonstop": 0, "1stop": 1, "2stop": 2})
)

In [13]:
cleaned["dep_hour"] = cleaned["departure"].dt.hour
cleaned["arr_hour"] = cleaned["arrival"].dt.hour

In [14]:
cleaned = cleaned.rename(columns={"from": "source", "to": "destination"})

In [15]:
# I am going to use 4 attributes for my 3 visualizations
cleaned = cleaned[["dep_hour", "arr_hour", "stops", "source", "destination"]]

In [16]:
# First 5 rows of data
cleaned.head()

,dep_hour,arr_hour,stops,source,destination
0,18,20,0,Delhi,Mumbai
1,19,21,0,Delhi,Mumbai
2,20,20,1,Delhi,Mumbai
3,21,23,1,Delhi,Mumbai
4,17,23,1,Delhi,Mumbai


In [17]:
# Convert data to CSV
file_path = os.path.join('data', 'data.csv')
cleaned.to_csv(file_path, index=False)

In [18]:
# Convert source and destination into JSON
unique_cities = pd.concat([cleaned["source"], cleaned["destination"]]).unique()
nodes_dict = {city: i for i, city in enumerate(unique_cities)}
nodes_list = [{"id": nodes_dict[city], "city": city} for city in nodes_dict.keys()]
links_df = cleaned.groupby(['source', 'destination']).size().reset_index(name='value')
links_df['source'] = links_df['source'].replace(nodes_dict)
links_df['destination'] = links_df['destination'].replace(nodes_dict)
links_df = links_df.rename(columns={'destination': 'target'})
links_list = links_df.to_dict(orient='records')
graph_data = {"nodes": nodes_list, "links": links_list}
graph_data

{'nodes': [{'id': 0, 'city': 'Delhi'},
  {'id': 1, 'city': 'Mumbai'},
  {'id': 2, 'city': 'Bangalore'},
  {'id': 3, 'city': 'Kolkata'},
  {'id': 4, 'city': 'Hyderabad'},
  {'id': 5, 'city': 'Chennai'}],
 'links': [{'source': 2, 'target': 5, 'value': 6410},
  {'source': 2, 'target': 0, 'value': 13756},
  {'source': 2, 'target': 4, 'value': 8971},
  {'source': 2, 'target': 3, 'value': 10029},
  {'source': 2, 'target': 1, 'value': 12940},
  {'source': 5, 'target': 2, 'value': 6493},
  {'source': 5, 'target': 0, 'value': 9783},
  {'source': 5, 'target': 4, 'value': 6103},
  {'source': 5, 'target': 3, 'value': 6983},
  {'source': 5, 'target': 1, 'value': 9338},
  {'source': 0, 'target': 2, 'value': 14012},
  {'source': 0, 'target': 5, 'value': 10780},
  {'source': 0, 'target': 4, 'value': 9328},
  {'source': 0, 'target': 3, 'value': 11934},
  {'source': 0, 'target': 1, 'value': 15291},
  {'source': 4, 'target': 2, 'value': 7898},
  {'source': 4, 'target': 5, 'value': 6395},
  {'source': 4, 

In [19]:
# Write JSON to filepath
file_path =  os.path.join('data', 'graph_data.json')

with open(file_path, 'w') as json_file:
    json.dump(graph_data, json_file)